#### Patient Arrival

To develop the process of patient arrival simulation, Perplexity LLM was used to create the initial code that could be adapted according to further requirements and refinement for the overall simulation process.  

Perplexity was provided with the following prompt:

**Patient Arrival to the Hospital Unit – Perplexity Prompt** 

*Main command* 

Code a discrete-event simulation model of a ward in python 3.10 and simpy 4. 

Please code the full model specified. Do not return a simplified version. Show all code. 

*General model logic*

All time units in the model are in hours. 

Each patient in the model has a unique identifier. The first patient to arrive has an identifier of 0. For each subsequent patient increment the identifier by 1. 

Patients should arrive and be assigned a gender at a ratio of 0.3384 male and 0.6613 female.  The patients should be assigned a ‘length of stay’ (LoS) with mean_LoS of 3, 7 or 12 days. 

On arrival, patients shall be allocated a patient_id starting at 1.  The patients should be generated using a function with a random seed of 42, and based on a np.random.poisson arrival_rate. 

When the patient data is created, it should be stored as a csv file, including the: 

Patient ID 

Day_Arrived 

Gender 

LoS 

 
After patients arrive they immediately leave the model. 

All patient types must have their own generator function 

The model should print out useful information after each event. 

The model should include a user settable run length. This defaults to 12 months. 


Simulation inputs 

The interarrival time distributions and parameters of patients are dependent on patient type. For each distribution time is measured in hours. 

* LoS 3 days = Exponential: 22.72 

* LoS 5 days  = Exponential: 26.0 

* LoS 7 days = Exponential: 37.0 

Simulation methodology 

numpy should be use for sampling. 

each interarrival distribution should have its own numpy.random.Generator object 



In [1]:
import simpy
import numpy as np
import csv
from datetime import datetime

In [2]:
# patient arrivals, generated by perplexity using a prepared prompt
# next step allocation to different beds on different wards
# this version now creates the CSV file at the end

class Hospital_Simulation:
    def __init__(self, run_length=8760):
        self.env = simpy.Environment()
        self.patient_counter = 0
        self.run_length = run_length  # Default 12 months in hours
        self.rng = np.random.default_rng(42)
        
        # Initialize patient type generators
        self.patient_types = [
            {'los_days': 3, 'interarrival_mean': 22.72, 'seed': 42},
            {'los_days': 5, 'interarrival_mean': 26.0, 'seed': 43},
            {'los_days': 7, 'interarrival_mean': 37.0, 'seed': 44}
        ]
        
        # Setup CSV logging

        self.csv_filename = 'hospital_patients.csv'
        

    def patient_generator(self, env, los_days, interarrival_mean, seed, writer):
        rng = np.random.default_rng(seed)
        while True:
            yield env.timeout(rng.exponential(interarrival_mean))
            self.patient_counter += 1
            patient_id = self.patient_counter
            
            # Assign patient attributes
            gender = 'male' if self.rng.random() < 0.3384 else 'female'
            los = los_days * 24  # Convert days to hours
            
            # Log patient data
            arrival_time = env.now
            writer.writerow([patient_id, f"{arrival_time:.2f}", gender, los])
            print(f"Patient {patient_id} ({gender}) arrived at {env.now:.2f}h with {los_days} day LoS")
            
            # Process patient stay
            yield env.timeout(los)
            print(f"Patient {patient_id} discharged after {los}h at {env.now:.2f}h")

    def run_simulation(self):
        # Open CSV file within context manager to ensure proper closure
        with open(self.csv_filename, 'w', newline='') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow(['Patient_ID', 'arrival_time', 'Gender', 'LoS'])

            # Start all patient generators
            for pt in self.patient_types:
                self.env.process(self.patient_generator(
                    self.env, 
                    pt['los_days'],
                    pt['interarrival_mean'],
                    pt['seed'],
                    writer  # Pass writer instance
                ))

            # Run simulation
            self.env.run(until=self.run_length)

# Example usage
if __name__ == '__main__':
    simulation = Hospital_Simulation()
    print("Starting hospital simulation...")
    simulation.run_simulation()
    print(f"Simulation completed. Data saved to {simulation.csv_filename}")    
   

Starting hospital simulation...
Patient 1 (female) arrived at 4.30h with 5 day LoS
Patient 2 (female) arrived at 13.71h with 7 day LoS
Patient 3 (female) arrived at 54.62h with 3 day LoS
Patient 1 discharged after 120h at 124.30h
Patient 4 (female) arrived at 126.54h with 5 day LoS
Patient 3 discharged after 72h at 126.62h
Patient 5 (male) arrived at 179.70h with 3 day LoS
Patient 2 discharged after 168h at 181.71h
Patient 6 (female) arrived at 199.66h with 7 day LoS
Patient 4 discharged after 120h at 246.54h
Patient 7 (female) arrived at 247.93h with 5 day LoS
Patient 5 discharged after 72h at 251.70h
Patient 8 (female) arrived at 305.88h with 3 day LoS
Patient 6 discharged after 168h at 367.66h
Patient 7 discharged after 120h at 367.93h
Patient 8 discharged after 72h at 377.88h
Patient 9 (male) arrived at 384.24h with 3 day LoS
Patient 10 (female) arrived at 398.90h with 5 day LoS
Patient 11 (female) arrived at 412.27h with 7 day LoS
Patient 9 discharged after 72h at 456.24h
Patient 